# 1.1 Cuda: Hello CPU and Hello GPU

In [ ]:
# To show that if we have the nvcc command
!which nvcc

/usr/local/cuda/bin/nvcc


In [ ]:
# To show that if there is cuda tookit installed
!ls /usr/local

bin    cuda	cuda-12.2  games	       include	lib64	   man	 share
colab  cuda-12	etc	   _gcs_config_ops.so  lib	licensing  sbin  src


In [ ]:
# To show the property of the nvidia card(On my one, I use the K80)
!nvidia-smi

Mon Jun 10 00:27:59 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   42C    P8              11W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [ ]:
!nvcc --list-gpu-arch

compute_50
compute_52
compute_53
compute_60
compute_61
compute_62
compute_70
compute_72
compute_75
compute_80
compute_86
compute_87
compute_89
compute_90


In [ ]:
#!rm hello.cu

In [ ]:
# Write a cu file contain the host and kernel code
%%writefile hello.cu

#include <cuda_runtime.h>
#include <stdio.h>
__global__ void hello(void)
{
  printf("GPU: Hello!\n");
}
int main(int argc,char **argv)
{
  printf("CPU: Hello!\n");
  hello<<<1,10>>>();
  cudaDeviceReset();
  return 0;
}

Overwriting hello.cu


In [ ]:
!pwd
!ls /content/

/content
arradd	arradd.cu  hello  hello.cu  numadd  numadd.cu  sample_data


In [ ]:
# Compile the code. The flag is needed if you use the Tesla K80.
!nvcc -arch=sm_75 -gencode=arch=compute_75,code=sm_75 hello.cu -o hello

In [ ]:
!./hello

CPU: Hello!
GPU: Hello!
GPU: Hello!
GPU: Hello!
GPU: Hello!
GPU: Hello!
GPU: Hello!
GPU: Hello!
GPU: Hello!
GPU: Hello!
GPU: Hello!


# 1.2 Cuda: Add Two Numbers

In [ ]:
%%writefile numadd.cu
// simple addition of two numbers
#include<stdio.h>
#include<cuda.h>
__global__ void numadd(int *a, int *b, int *c) { //kernel definition
  *c = *a + *b;
}

int main() {
    int a, b, c;           // host copies pf variables a, b, and c
    int *d_a, *d_b, *d_c;  // device copies of variables a, b, and c
    int size = sizeof (int);
    // Alocated space for device copies copies of a, b, c
    cudaMalloc((void **)&d_a, size);
    cudaMalloc((void **)&d_b, size);
    cudaMalloc((void **)&d_c, size);
    // setup input values
    c = 0; a = 3; b = 5;
    // copy inputs to device
    cudaMemcpy(d_a, &a, size, cudaMemcpyHostToDevice);
    cudaMemcpy(d_b, &b, size, cudaMemcpyHostToDevice);
    // Add two numbers at the GPU device
    // 1: number of blocks, 1: number of thread per block/
    numadd<<<1, 1>>>(d_a, d_b, d_c);
    // Here we are copying content from GPU(Device) to CPU(Host)
    cudaMemcpy(&c, d_c, size, cudaMemcpyDeviceToHost);
    printf("Sum = %d\n", c);

    /* Free the memory allocated to pointers d,e,f */
    cudaFree(d_a);
    cudaFree(d_b);
    cudaFree(d_c);
    return 0;
}

Overwriting numadd.cu


In [ ]:
#!rm add.cu
!ls

arradd	arradd.cu  hello  hello.cu  numadd.cu  sample_data


In [ ]:
# Compile the code. The flag is needed if you use the Tesla K80.
!nvcc -arch=sm_75 -gencode=arch=compute_75,code=sm_75 numadd.cu -o numadd

In [ ]:
!./numadd

Sum = 8


# 1.3 Cuda: Addition of 2 1-D Arrays

In [ ]:
# Write a cu file contain the host and kernel code
%%writefile arradd.cu
#include<stdio.h>
#include<cuda.h>
__global__ void arradd(int *x,int *y, int *z) { //kernel definition
  int id=blockIdx.x;
  /* blockIdx.x gives the respective block id which starts from 0 */
  z[id]=x[id]+y[id];
}

int main() {
    // we add two arrays a[6] + b [6] into c[6]
    int a[6] = {1, 2, 3, 4, 5, 6};
    int b[6] = {2, 3, 4, 5, 6, 7};
    int c[6];
    int *d,*e,*f; int i;
    // we do not take input from user.
    /*
    printf("\n Enter six elements of first array\n");
    for(i=0;i<6;i++) {
      scanf("%d",&a[i]);
    }
    printf("\n Enter six elements of second array\n");
    for(i=0;i<6;i++) {
      scanf("%d",&b[i]);
    }
    */

    /* cudaMalloc() allocates memory from Global memory on GPU */
    cudaMalloc((void **)&d,6*sizeof(int)); cudaMalloc((void **)&e,6*sizeof(int));
    cudaMalloc((void **)&f,6*sizeof(int));

    /* cudaMemcpy() copies the contents from destination to source.
     Here destination is GPU(d,e) and source is CPU(a,b) */
    cudaMemcpy(d,a,6*sizeof(int),cudaMemcpyHostToDevice);
    cudaMemcpy(e,b,6*sizeof(int),cudaMemcpyHostToDevice);

    /* Here we are copying content from GPU(Device) to CPU(Host) */
    /* call to kernel. Here 6 is number of blocks,
      1 is the number of threads per block and d,e,f are the arguments */
    arradd<<<6,1>>>(d,e,f);

    /* Here we are copying content from GPU(Device) to CPU(Host) */
    cudaMemcpy(c,f,6*sizeof(int),cudaMemcpyDeviceToHost);
    printf("\nSum of two arrays:\n ");
    for(i=0;i<6;i++) {
      printf("%d\t",c[i]);
    }

    /* Free the memory allocated to pointers d,e,f */
    cudaFree(d);
    cudaFree(e);
    cudaFree(f);
    return 0;
}

Overwriting arradd.cu


In [ ]:
# Compile the code. The flag is needed if you use the Tesla K80.
!nvcc -arch=sm_75 -gencode=arch=compute_75,code=sm_75 arradd.cu -o arradd

In [ ]:
!ls

arradd	arradd.cu  hello  hello.cu  sample_data


In [ ]:
!./arradd


Sum of two arrays:
 3	5	7	9	11	13	

# 1.4 Find GPU Capacity

In [3]:
# Write a cu file contain the host and kernel code
%%writefile gpucap.cu
#include<stdio.h>
#include<cuda.h>

int main()
{
    cudaDeviceProp p;
    int device_id;
    int major;
    int minor;


    cudaGetDevice(&device_id);
    cudaGetDeviceProperties(&p,device_id);

    major=p.major;
    minor=p.minor;

    printf("Name of GPU on your system is %s\n",p.name);

    printf("\n Compute Capability of a current GPU on your system is %d.%d",major,minor);

    return 0;
}

Writing gpucap.cu


In [4]:
# Compile the code. The flag is needed if you use the Tesla K80.
!nvcc -arch=sm_75 -gencode=arch=compute_75,code=sm_75 gpucap.cu -o gpucap

In [5]:
!ls

gpucap	gpucap.cu  sample_data


In [6]:
!./gpucap

Name of GPU on your system is 

 Compute Capability of a current GPU on your system is -1414679944.32767